In [1]:
# Import packages.
import pandas as pd
import json


In [2]:
"""References and documentation.
Covid Hesitancy: US Department of Health and Human Services -
    https://aspe.hhs.gov/reports/vaccine-hesitancy-covid-19-state-county-local-estimates

Covid Cases and Deaths: John Hopkins University -
    https://github.com/CSSEGISandData/COVID-19
    
Population, Education, Income, and Unemployment: US Department of Agriculture, Economic Research Service
    https://www.ers.usda.gov/data-products/county-level-data-sets/download-data/
"""


'References and documentation.\nCovid Hesitancy: US Department of Health and Human Services -\n    https://aspe.hhs.gov/reports/vaccine-hesitancy-covid-19-state-county-local-estimates\n\nCovid Cases and Deaths: John Hopkins University -\n    https://github.com/CSSEGISandData/COVID-19\n    \nPopulation, Education, Income, and Unemployment: US Department of Agriculture, Economic Research Service\n    https://www.ers.usda.gov/data-products/county-level-data-sets/download-data/\n'

In [3]:
# Define global variables.
PATH = '/Users/justinsima/dir/projects/portfolio/eda/vaccine_hesitancy/'
COVID_REPO = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'


In [4]:
# Load Covid hesitancy data.
data_path = PATH + 'data/vaccine_hesitancy.csv'
df = pd.read_csv(data_path)

df.head(3)


,FIPS Code,County Name,State,Estimated hesitant,Estimated hesitant or unsure,Estimated strongly hesitant,Social Vulnerability Index (SVI),SVI Category,CVAC level of concern for vaccination rollout,CVAC Level Of Concern,...,Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,Percent non-Hispanic Native Hawaiian/Pacific Islander,Percent non-Hispanic White,Geographical Point,State Code,County Boundary,State Boundary
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.2400,0.1383,0.89,Very High Vulnerability,0.64,High Concern,...,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-85.841259 33.104456, -85.8409...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.2350,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,...,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-86.303069 33.46316, -86.30306...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,...,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.52534299999999 32.132773, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."


In [5]:
# Display features and their datatypes.
data_types = df.dtypes

for ind, dtype in data_types.iteritems():
    print(f'Feature: {ind}, \ndata type: {dtype}.\n')


Feature: FIPS Code, 
data type: int64.

Feature: County Name, 
data type: object.

Feature: State, 
data type: object.

Feature: Estimated hesitant, 
data type: float64.

Feature: Estimated hesitant or unsure, 
data type: float64.

Feature: Estimated strongly hesitant, 
data type: float64.

Feature: Social Vulnerability Index (SVI), 
data type: float64.

Feature: SVI Category, 
data type: object.

Feature: CVAC level of concern for vaccination rollout, 
data type: float64.

Feature: CVAC Level Of Concern, 
data type: object.

Feature: Percent adults fully vaccinated against COVID-19 (as of 6/10/21), 
data type: float64.

Feature: Percent Hispanic, 
data type: float64.

Feature: Percent non-Hispanic American Indian/Alaska Native, 
data type: float64.

Feature: Percent non-Hispanic Asian, 
data type: float64.

Feature: Percent non-Hispanic Black, 
data type: float64.

Feature: Percent non-Hispanic Native Hawaiian/Pacific Islander, 
data type: float64.

Feature: Percent non-Hispanic White

In [6]:
# Filter data to columns of interest.
columns = [
    "State",
    "County Name",
    "Estimated hesitant",
    "Estimated hesitant or unsure",
    "Estimated strongly hesitant",
    "Percent adults fully vaccinated against COVID-19 (as of 6/10/21)",
    "Percent Hispanic",
    "Percent non-Hispanic American Indian/Alaska Native",
    "Percent non-Hispanic Asian",
    "Percent non-Hispanic Black",
    "Percent non-Hispanic Native Hawaiian/Pacific Islander",
    "Percent non-Hispanic White"
]

df = df[columns]

# Reformat columns.
df['State'] = df['State'].str.lower()

# Rename some columns.
df = df.rename(columns={
    "Estimated hesitant":'hesitant',
    "Estimated hesitant or unsure":'hesitant_unsure',
    "Estimated strongly hesitant":'strongly_hesitant'
})

# Add only hesitant column.
df['unsure'] = df['hesitant_unsure'] - df['hesitant']


In [7]:
%%time
# Load John Hopkin's University data containing covid cases and deaths by county.
# Function to load and process csv files from this data source.
def import_jhu(file_name:str, prefix:str):

    # Define columns to keep.
    columns_keep = [
        'Combined_Key',
        '1/22/20',
        '1/1/21',
        '6/10/21',
        '2/25/22'
    ]
    
    # Load data.
    new_path = COVID_REPO + file_name
    df_ = pd.read_csv(new_path, usecols=columns_keep)
    
    # Reformat key to match original data.
    df_['Combined_Key'] = df_['Combined_Key'].str.strip(', US').str.replace(', ', ' County, ')
    
    # Rename columns and set 'Combined_Key' as an index.
    df_ = df_[columns_keep].rename(columns={
        '1/22/20':f'{prefix}_1/22/20',
        '1/1/21':f'{prefix}_1/1/21',
        '6/10/21':f'{prefix}_6/10/21',
        '2/25/22':f'{prefix}_2/25/22'
    }).set_index('Combined_Key')
    
    return df_

# Load and process confirmed cases and deaths by county.
df_cases = import_jhu('time_series_covid19_confirmed_US.csv', 'cases')
df_deaths = import_jhu('time_series_covid19_deaths_US.csv', 'deaths')


CPU times: user 222 ms, sys: 165 ms, total: 387 ms
Wall time: 5.96 s


In [8]:
df_cases.head(3)


,cases_1/22/20,cases_1/1/21,cases_6/10/21,cases_2/25/22
Combined_Key,,,,
"Autauga County, Alabama",0,4239,7211,15479
"Baldwin County, Alabama",0,13823,21765,54904
"Barbour County, Alabama",0,1517,2345,5438


In [9]:
df_deaths.head(3)


,deaths_1/22/20,deaths_1/1/21,deaths_6/10/21,deaths_2/25/22
Combined_Key,,,,
"Autauga County, Alabama",0,50,113,194
"Baldwin County, Alabama",0,169,313,640
"Barbour County, Alabama",0,33,59,93


In [10]:
# Join death and confirmed cases dataframes to original data.
df = df.join(df_cases, on='County Name', how='left')
df = df.join(df_deaths, on='County Name', how='left')

# Create list of counties that were missing case and death information.
counties_missing_data = [county for county in df['County Name'].loc[df['cases_1/22/20'].isna()]]

df.head(3)


,State,County Name,hesitant,hesitant_unsure,strongly_hesitant,Percent adults fully vaccinated against COVID-19 (as of 6/10/21),Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,...,Percent non-Hispanic White,unsure,cases_1/22/20,cases_1/1/21,cases_6/10/21,cases_2/25/22,deaths_1/22/20,deaths_1/1/21,deaths_6/10/21,deaths_2/25/22
0,alabama,"Tallapoosa County, Alabama",0.1806,0.2400,0.1383,0.305,0.0242,0.0022,0.0036,0.2697,...,0.6887,0.0594,0.0,2383.0,4114.0,11288.0,0.0,98.0,155.0,221.0
1,alabama,"Talladega County, Alabama",0.1783,0.2350,0.1368,0.265,0.0229,0.0043,0.0061,0.3237,...,0.6263,0.0567,0.0,5211.0,8439.0,21952.0,0.0,76.0,183.0,350.0
2,alabama,"Wilcox County, Alabama",0.1735,0.2357,0.1337,0.394,0.0053,0.0009,0.0003,0.6938,...,0.2684,0.0622,0.0,890.0,1270.0,2855.0,0.0,19.0,30.0,42.0


In [11]:
%%time
# Load USDA data.
# Function to load and process excel files from this data source.
def import_usda(file_name:str, header:int, relevant_cols:list, rename_dict:dict=None):
    # Define needed columns.
    use_cols = relevant_cols + ['State', 'Area name']
    keep_cols = relevant_cols + ['Key']
    
    # Load data from directory.
    df_path = PATH + file_name
    df_ = pd.read_excel(df_path, header=header, usecols=use_cols)

    # Filter out non-county data.
    df_ = df_.loc[df_['Area name'].str.contains('County', case=False, na=False)]
    
    # Load json containing state names and their codes.
    states_path = PATH + 'data/states.json'
    with open(states_path) as json_file:
        states_list = json.load(json_file)

    # Create dictionary of state codes and names.
    states_dict = {}
    for state in states_list:
        state_name = state['State']
        state_code = state['Code']

        states_dict[state_code] = state_name

    # Don't forget Puerto Rico.
    states_dict['PR'] = 'Puerto Rico'

    # Map state codes to state names.
    df_['State'] = df_['State'].map(states_dict).copy()
    
    # Reformat 'Area name' if it isn't correctly formatted.
    df_['Area name'] = df_['Area name'].str.split(',').str[0]

    # Add key to join to df.
    df_['Key'] = df_['Area name'] + ', ' + df_['State']

    # Keep only relevant columns.
    df_ = df_.loc[:,keep_cols]

    # Set index.
    df_ = df_.set_index('Key')
    
    # If rename dict is passed, apply now.
    if rename_dict:
        df_ = df_.rename(columns=rename_dict)
    
    return df_


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 2.86 µs


In [12]:
# Create dataframe of population data.
df_pop = import_usda('data/PopulationEstimates.xlsx', 1, ['Population 2020'], {'Population 2020':'population'})

# Create dataframe of education data.
df_ed = import_usda('data/Education.xls',
                    4,
                    [
                        "Percent of adults with less than a high school diploma, 2015-19",
                        "Percent of adults with a high school diploma only, 2015-19",
                        "Percent of adults completing some college or associate's degree, 2015-19",
                        "Percent of adults with a bachelor's degree or higher, 2015-19"
                    ],
                    {
                        "Percent of adults with less than a high school diploma, 2015-19":'percent_no_highschool',
                        "Percent of adults with a high school diploma only, 2015-19":'percent_highschool',
                        "Percent of adults completing some college or associate's degree, 2015-19":'percent_some_colllege',
                        "Percent of adults with a bachelor's degree or higher, 2015-19":'percent_college'
                    }
                   )

# Create dataframe of unemployment and income data.
df_emp = import_usda('data/Unemployment.xlsx',
                     4,
                     [
                         'Unemployment_rate_2020',
                         'Unemployment_rate_2019',
                         'Median_Household_Income_2019'
                     ],
                     {
                         'Unemployment_rate_2020':'unemployment_2020',
                         'Unemployment_rate_2019':'unemployment_2019',
                         'Median_Household_Income_2019':'mhi_2019'
                     }
                    )


In [13]:
# Take a look at one of the dataframes.
df_emp


,unemployment_2020,unemployment_2019,mhi_2019
Key,,,
"Autauga County, Alabama",4.9,2.7,58233.0
"Baldwin County, Alabama",5.6,2.8,59871.0
"Barbour County, Alabama",7.0,3.8,35972.0
"Bibb County, Alabama",6.6,3.1,47918.0
"Blount County, Alabama",4.1,2.7,52902.0
...,...,...,...
"Sweetwater County, Wyoming",7.4,4.0,80639.0
"Teton County, Wyoming",6.0,2.8,98837.0
"Uinta County, Wyoming",6.3,4.0,70756.0


In [14]:
# Append new columns to dataframe.
df = df.join(df_pop, on='County Name', how='left')
df = df.join(df_ed, on='County Name', how='left')
df = df.join(df_emp, on='County Name', how='left')

df.head()


,State,County Name,hesitant,hesitant_unsure,strongly_hesitant,Percent adults fully vaccinated against COVID-19 (as of 6/10/21),Percent Hispanic,Percent non-Hispanic American Indian/Alaska Native,Percent non-Hispanic Asian,Percent non-Hispanic Black,...,deaths_6/10/21,deaths_2/25/22,population,percent_no_highschool,percent_highschool,percent_some_colllege,percent_college,unemployment_2020,unemployment_2019,mhi_2019
0,alabama,"Tallapoosa County, Alabama",0.1806,0.2400,0.1383,0.305,0.0242,0.0022,0.0036,0.2697,...,155.0,221.0,41311.0,19.346287,32.706596,29.504309,18.442808,6.9,3.0,47100.0
1,alabama,"Talladega County, Alabama",0.1783,0.2350,0.1368,0.265,0.0229,0.0043,0.0061,0.3237,...,183.0,350.0,82149.0,19.670317,33.720287,31.648600,14.960798,7.0,3.4,47719.0
2,alabama,"Wilcox County, Alabama",0.1735,0.2357,0.1337,0.394,0.0053,0.0009,0.0003,0.6938,...,30.0,42.0,10600.0,23.545122,39.471466,24.486927,12.496486,14.7,7.2,30998.0
3,alabama,"Washington County, Alabama",0.1735,0.2357,0.1337,0.308,0.0146,0.0731,0.0025,0.2354,...,39.0,55.0,15388.0,17.371761,43.063633,26.899347,12.665257,8.0,4.7,48864.0
4,alabama,"Winston County, Alabama",0.1805,0.2313,0.1379,0.163,0.0315,0.0034,0.0016,0.0073,...,73.0,120.0,23540.0,21.187923,38.214035,27.768711,12.829330,4.7,3.3,40827.0


In [15]:
# Check for any new counties with missing data.
new_counties_missing = [county for county in df['County Name'].loc[df.population.isna()]
                            if county not in counties_missing_data]

if new_counties_missing:
    print('New counties are now missing data.')
if not new_counties_missing:
    print('No new counties are missing data.')


No new counties are missing data.


In [16]:
# Define lists grouping racial groups as a minority or non-minority in America.
# Reference: https://minorityrights.org/country/united-states-of-america/
american_racial_minorities = [
    'Percent Hispanic',
    'Percent non-Hispanic American Indian/Alaska Native',
    'Percent non-Hispanic Asian',
    'Percent non-Hispanic Black',
    'Percent non-Hispanic Native Hawaiian/Pacific Islander'
]

american_racial_nonminorities = [
    'Percent non-Hispanic White'
]

provided_racial_groups = american_racial_minorities + american_racial_nonminorities


In [17]:
# Create percent minority column.
df['Percent Minority'] = 0

for col in american_racial_minorities:
    df['Percent Minority'] = df['Percent Minority'] + df[col]


In [18]:
# Create indicator for whether or not over 50% of a county's population belongs to a racial minority group.
df['Majority Minority County'] = (df['Percent Minority'] > 0.5).map({False:0, True:1})


In [19]:
# Add deaths per case columns.
df['deaths_per_case_1/22/20'] = df['deaths_1/22/20'] / df['cases_1/22/20']
df['deaths_per_case_1/1/21'] = df['deaths_1/1/21'] / df['cases_1/1/21']
df['deaths_per_case_6/10/21'] = df['deaths_6/10/21'] / df['cases_6/10/21']
df['deaths_per_case_2/25/22'] = df['deaths_2/25/22'] / df['cases_2/25/22']


In [20]:
# Create additional 'percapita' columns.   
columns_cases_deaths = [
    'cases_1/22/20',
    'cases_1/1/21',
    'cases_6/10/21',
    'cases_2/25/22',
    'deaths_1/22/20',
    'deaths_1/1/21',
    'deaths_6/10/21',
    'deaths_2/25/22'
]

columns_hesitancy = [
    'hesitant',
    'hesitant_unsure',
    'strongly_hesitant'
]

columns_append = columns_cases_deaths + columns_hesitancy

# Iterate through columns of cases and deaths, and append a per capita columns.
columns_per_capita = []
for col in columns_append:
    new_colname = 'percapita_' + col
    df[new_colname] = df[col] / df['population']
    
    columns_per_capita.append(new_colname)

columns_per_capita_cases = [
    'percapita_cases_1/22/20',
    'percapita_cases_1/1/21',
    'percapita_cases_6/10/21',
    'percapita_cases_2/25/22'
]
    
columns_per_capita_deaths = [
    'percapita_deaths_1/22/20',
    'percapita_deaths_1/1/21',
    'percapita_deaths_6/10/21',
    'percapita_deaths_2/25/22'
]


In [21]:
# Rename columns to be more interpretable.
df = df.rename(columns={'hesitant':'Percent Hesitant',
    'hesitant_unsure':'Percent Hesitant or Unsure',
    'strongly_hesitant':'Percent Strongly Hesitant',
    'unsure':'Percent Unsure',
    'cases_1/22/20':'Cases (1/22/20)',
    'cases_1/1/21':'Cases (1/1/21)',
    'cases_6/10/21':'Cases (6/10/21)',
    'cases_2/25/22':'Cases (2/25/22)',
    'deaths_1/22/20':'Deaths (1/22/20)',
    'deaths_1/1/21':'Deaths (1/1/21)',
    'deaths_6/10/21':'Deaths (6/10/21)',
    'deaths_2/25/22':'Deaths (2/25/22)',
    'percent_no_highschool':'Percent No Highschool Education',
    'percent_highschool':'Percent Highschool Education',
    'percent_some_colllege':'Percent Some College Education',
    'percent_college':'Percent College Education',
    'unemployment_2020':'Unemployment Rate 2020',
    'unemployment_2019':'Unemployment Rate 2019',
    'mhi_2019':'Median Household Income 2019',
    'deaths_per_case_1/22/20':'Deaths Per Case (1/22/20)',
    'deaths_per_case_1/1/21':'Deaths Per Case (1/1/21)',
    'deaths_per_case_6/10/21':'Deaths Per Case (6/10/21)',
    'deaths_per_case_2/25/22':'Deaths Per Case (2/25/22)',
    'percapita_cases_1/22/20':'Cases Per Capita (1/22/20)',
    'percapita_cases_1/1/21':'Cases Per Capita (1/1/21)',
    'percapita_cases_6/10/21':'Cases Per Capita (6/10/21)',
    'percapita_cases_2/25/22':'Cases Per Capita (2/25/22)',
    'percapita_deaths_1/22/20':'Covid Deaths Per Capita (1/22/20)',
    'percapita_deaths_1/1/21':'Covid Deaths Per Capita (1/1/21)',
    'percapita_deaths_6/10/21':'Covid Deaths Per Capita (6/10/21)',
    'percapita_deaths_2/25/22':'Covid Deaths Per Capita (2/25/22)'}
)

In [22]:
# Save dataframe as pkl.
save_path = PATH + 'data/data.pkl'
df.to_pickle(save_path)
